In [1]:
## Importamos librerias requeridas para el EDA
import numpy as np
import pandas as pd

In [2]:
## Leer df

path = 'data/reddit_comments_160k.csv'
df = pd.read_csv(path)
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
df['hateResult'] = df['toxic'] | df['severe_toxic'] | df['obscene'] | df['threat'] | df['insult'] | df['identity_hate']
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,hateResult
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0,0


In [4]:
df.shape

(159571, 9)

In [5]:
df.drop(['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',	'identity_hate'], axis = 'columns', inplace=True)
df

,comment_text,hateResult
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


# Data Preprocessing

# Data Transformations

In [6]:
from sklearn.feature_extraction.text import CountVectorizer        # Transforms text into a sparse matrix of n-gram counts
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer       # Convert a collection of raw documents to a matrix of TF-IDF features

vect= TfidfVectorizer(max_features=5000,stop_words='english')
train_vec=vect.fit_transform(df.comment_text)
test_vec=vect.fit_transform(df.comment_text)

In [7]:
y = df.iloc[:, -1].values

# Entrenar

In [8]:
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

# Get metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from sklearn.model_selection import train_test_split

#Usamos tuberias
from sklearn.pipeline import Pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df['comment_text'], y, test_size = 0.20, random_state = 0)

### Creamos una tuberia con el modelo de CatBoosting

In [10]:
pipeline = Pipeline([
    ('bow', TfidfVectorizer(max_features=5000,stop_words='english')),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier_lg', LogisticRegression(solver='lbfgs')),  # train on TF-IDF vectors w/ Logistic Regression classifier
])

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('bow',
                 TfidfVectorizer(max_features=5000, stop_words='english')),
                ('tfidf', TfidfTransformer()),
                ('classifier_lg', LogisticRegression())])

In [13]:
y_pred= pipeline.predict(X_test)

In [19]:
# Accuracy del modelo
acc = accuracy_score(y_test, y_pred)
# Overfitting
y_pred_train = pipeline.predict(X_train)
acc_train = accuracy_score(y_train, y_pred_train)

print("\n" + "LogisticRegression")

print( "classification_report\n\n", classification_report(y_test, y_pred))
#print( "classification_report_train\n\n", classification_report(y_train, y_pred_train))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

print(f"Accuaracy: {acc}")
print(f"Accuaracy_train: {acc_train}")
print(cm)
test_y_prob = pipeline.predict_proba(X_test)[:,1]


LogisticRegression
classification_report

               precision    recall  f1-score   support

           0       0.96      0.99      0.98     28610
           1       0.92      0.61      0.74      3305

    accuracy                           0.95     31915
   macro avg       0.94      0.80      0.86     31915
weighted avg       0.95      0.95      0.95     31915

Accuaracy: 0.9546294845683848
Accuaracy_train: 0.9586545089929185
[[28444   166]
 [ 1282  2023]]


# Guardamos nuestro Bow y nuestro mejor modelo

In [20]:
import pickle

In [21]:
### Guardando diccionario BoW para usarlo mas tarde en la predicción
bow_path = 'BoW_HateSpeech_Model_LogisticRegression.pkl'
pickle.dump(train_vec, open(bow_path, "wb"))

In [22]:
pickle.dump(pipeline, open('logisticRegression_model_more_data_trained', 'wb'))